In [93]:
%matplotlib inline
from pandas_datareader import data
import pandas as pd
import random
from SALib.sample import latin
from functions.stylizedfacts import *
from functions.evolutionaryalgo import *
from pandas_datareader import data
from functions.helpers import hurst, organise_data, div_by_hundred, discounted_value_cash_flow, find_horizon, calculate_npv
import matplotlib.pyplot as plt
import quandl
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

# Bootstrapped moments S&P 500
Following the procedure presented by [Franke & Westerhoff (2012)](https://www.sciencedirect.com/science/article/pii/S0165188912000802).

## 1 Get data

In [81]:
start_date = '1993-03-10'
end_date = '2018-01-01'
#window = 320

spy_nom_price = data.DataReader("SPY", 
                       start=start_date, 
                       end=end_date, 
                       data_source='yahoo')['Close']
spy_nom_returns = spy_nom_price.pct_change()[1:]

spy_real_price = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH", authtoken="8EKNms1cLLU-dBjk5Y4d").loc[start_date:end_date]
spy_real_price = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH", authtoken="8EKNms1cLLU-dBjk5Y4d").loc[start_date:end_date]
spy_real_price = spy_real_price.rename(columns={"Value": "price"})

spy_returns = spy_real_price.pct_change()[1:]

spy_dividends = quandl.get("BUNDESBANK/BBQFS_M_US_CORP_PRICE_SP500__X_0000", authtoken="8EKNms1cLLU-dBjk5Y4d").loc[start_date:end_date]
spy_dividends = spy_dividends.rename(columns={"Value": "dividends"})
spy_dividends.index = spy_dividends.index + pd.offsets.MonthBegin(1)
spy_all = spy_dividends.join(spy_real_price)

spy_t_bill = quandl.get("FRED/INTGSTUSM193N", authtoken="8EKNms1cLLU-dBjk5Y4d").loc[start_date:end_date]
spy_t_bill = spy_t_bill.rename(columns={"Value": "dividends"})

spy_all['t_bill_int'] = spy_t_bill / 100

spy_all['div_growth'] = spy_all['dividends'].pct_change()
spy_all['av_div_growth'] = spy_all['div_growth'].rolling(window=window,center=False).mean()
spy_all['yield'] = spy_all['dividends'].iloc[1:] / spy_all['price'].iloc[:-1]
spy_all['ml_discount'] = [0.052 for x in range(len(spy_all))]
spy_all['fixed_yield'] = spy_all['t_bill_int'] + spy_all['ml_discount']

#spy_all = spy_all.iloc[-window:-1]

spy_all['fundamental'] = ((1 + spy_all['av_div_growth']) / (spy_all['fixed_yield'] - spy_all['av_div_growth'])) * spy_all['dividends']
spy_all['dev_fundamental'] = (spy_all['price'] - spy_all['fundamental']) / spy_all['fundamental']

## 2 Bootstrap data

In [82]:
# short block lenght = 250
len(spy_nom_returns)

6250

In [83]:
block_size = 250
# subdivide data in blocks of block_size

In [100]:
data_blocks = []
for x in range(0, len(spy_nom_returns), block_size):
    data_blocks.append(list(spy_nom_returns[x:x+block_size]))

In [105]:
# draw 5000 random series
bootstrapped_series = []
for i in range(5000):
    sim_data = [random.choice(data_blocks) for _ in data_blocks]
    sim_data2 = [j for i in sim_data for j in i]
    bootstrapped_series.append(sim_data2)

In [106]:
# now we can calculate the moments. Mean and standard deviations. 

# then we can minimize the model average deviations. 

5000

## 3 Choose moments

I use the following moments of returns: 

1. mean first-order autocorrelation of the raw returns (no predictability),
2. autocorrelations at lags t ¼ 1
3. autocorrelations at lags t ¼ 5
4. mean first-order autocorrelation of the of the absolute returns (volatility clustering),
5. Kurtosis (fat tails), 

and for long memory:
 
6. the autocorrelation function of vt ¼ at lag t ¼ 10
7. the autocorrelation function of vt ¼ at lag t 25,
8. the autocorrelation function of vt ¼ at lag t50; 
9. the autocorrelation function of vt ¼ at lag t 100.

Finally, for to measure the deviation from fundamentals, I use the 

In [30]:
spy_autocorrelation = autocorrelation_returns(spy_returns['price'], 25)
spy_autocorrelation1 = spy_returns['price'].autocorr(lag=1)
spy_autocorrelation5 = spy_returns['price'].autocorr(lag=5)
spy_kurtosis = kurtosis(spy_returns['price'])
spy_autocorrelation_abs = autocorrelation_abs_returns(spy_returns['price'], 25)
#spy_hurst = hurst(spy_all['price'])

# long memory:
spy_abs_auto10 = spy_returns['price'].abs().autocorr(lag=10)
spy_abs_auto25 = spy_returns['price'].abs().autocorr(lag=25)
spy_abs_auto50 = spy_returns['price'].abs().autocorr(lag=50)
spy_abs_auto100 = spy_returns['price'].abs().autocorr(lag=100)

#spy_dev_fund_hurst = hurst(spy_all['dev_fundamental'].iloc[:-1])
av_def_fund = abs(np.mean(spy_all['dev_fundamental']))
stylized_facts_spy = [spy_autocorrelation, spy_autocorrelation1, spy_autocorrelation5,
                      spy_kurtosis, spy_autocorrelation_abs, spy_abs_auto10, 
                      spy_abs_auto25, spy_abs_auto50, spy_abs_auto100, av_def_fund]

stylized_facts_spy = pd.DataFrame(stylized_facts_spy, columns=['S&P500'], 
             index=['autocorrelation', 'autocorrelation1', 'autocorrelation5',
                    'kurtosis', 'autocorrelation_abs', 'acf_abs10', 'acf_abs25',
                    'acf_abs50', 'acf_abs100', 'av_dev_from_fund']).to_dict()['S&P500']

In [31]:
stylized_facts_spy

{'acf_abs10': 0.072475104895203979,
 'acf_abs100': 0.022149454371627148,
 'acf_abs25': -0.042522722517340086,
 'acf_abs50': 0.01370050605199658,
 'autocorrelation': 0.00030320061625216735,
 'autocorrelation1': 0.24449426927505946,
 'autocorrelation5': 0.091815678384919108,
 'autocorrelation_abs': 0.049914003700974562,
 'av_dev_from_fund': 1.9880606822750695,
 'kurtosis': 3.0676490833426238}